In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout, Dense
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

In [7]:
df = pd.read_csv('Downloads/creditcard_2023.csv')

In [8]:
scaler = MinMaxScaler()
previsores = df.iloc[:, : -1].values
previsores = scaler.fit_transform(previsores)

In [9]:
classe = df.iloc[:, -1].values

In [10]:
x_train, x_test, y_train, y_test = train_test_split(previsores, classe, test_size = 0.5, random_state=0)

In [11]:
feature_model = ExtraTreesClassifier()
feature_model.fit(x_train, y_train)

ExtraTreesClassifier()

In [12]:
importance_scores = feature_model.feature_importances_

num_top_features = 10

In [13]:
top_feature_indices = np.argsort(importance_scores)[::-1][:num_top_features]
top_feature_indices

array([ 0,  4, 17, 11, 12, 14,  3, 10,  1, 16], dtype=int64)

In [14]:
x_train = x_train[:, top_feature_indices]
x_test = x_test[:, top_feature_indices]

In [17]:
def create_model(neurons=64, layers=2, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_dim=x_train.shape[1]))
    
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
        model.add(Dropout(dropout_rate))
     
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Defina os hiperparâmetros que você deseja testar
neurons = [32, 64, 128]
layers = [1, 2, 3]
dropout_rates = [0.2, 0.3, 0.4]

best_accuracy = 0.0
best_hyperparameters = {}

for n in neurons:
    for l in layers:
        for d in dropout_rates:
            # Crie um modelo Keras com os hiperparâmetros atuais
            model = create_model(neurons=n, layers=l, dropout_rate=d)
            model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=0)
            
            # Faça previsões no conjunto de teste
            y_pred = model.predict(x_test)
            y_pred_classes = (y_pred > 0.5)
            
            # Calcule a precisão
            accuracy = accuracy_score(y_test, y_pred_classes)
            
            # Verifique se a precisão é a melhor até agora
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_hyperparameters = {'neurons': n, 'layers': l, 'dropout_rate': d}

# Exiba os melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros:", best_hyperparameters)
print("Melhor precisão:", best_accuracy)

8885/8885 [==============================] - 23s 3ms/step
Melhores hiperparâmetros: {'neurons': 128, 'layers': 3, 'dropout_rate': 0.4}
Melhor precisão: 0.9995111056398712
